In [4]:
import pandas as pd
import numpy as np
from datetime import datetime
import os 
from datapipeline import DataFeed

In [5]:
from models import BrazilCFR

In [6]:
self = BrazilCFR()

In [7]:
self.base_prediction

Brazil: $888.36 ->> $1068.46 | 20.3% up


1068.462332725581

In [ ]:
def base_prediction(self):
    """
    Predicts next months CFR pricing for brazil
    """
    # Food Price Index with lags
    fao_2 = self.FAOPriceIndex_2 * self.get_food_price_index['Food Price Index'].iloc[-1-2]
    fao_3 = self.FAOPriceIndex_3 * self.get_food_price_index['Food Price Index'].iloc[-1-3]
    fao_6 = self.FAOPriceIndex_6 * self.get_food_price_index['Food Price Index'].iloc[-1-6]
    # Exchange rates
    eurusd = self.USDEURO * (self.get_eurusd['Adj Close']).iloc[-1]
    # Fertilizer production quadratic
    fertprodquad = self.FertProdQuad * (self.get_total_fertilizer_production['Total Fertilizer Production']**2).iloc[-1]
    # G20Inflation
    g20inflation = self.G20Inflation * self.get_g20_cpi['G20CPI'].dropna().iloc[-1]
    # DUMMY
    dm2 = 1*self.dm2 if self.month == 2 else 0
    dm3 = 1*self.dm3 if self.month == 3 else 0
    dm4 = 1*self.dm4 if self.month == 4 else 0
    dm5 = 1*self.dm5 if self.month == 5 else 0
    dm6 = 1*self.dm6 if self.month == 6 else 0
    dm7 = 1*self.dm7 if self.month == 7 else 0
    dm8 = 1*self.dm8 if self.month == 8 else 0
    dm9 = 1*self.dm9 if self.month == 9 else 0
    dm10 =1*self.dm10 if self.month == 10 else 0
    dm11 =1*self.dm11 if self.month == 11 else 0
    dm12 =1*self.dm12 if self.month == 12 else 0
    monthly_dummy = dm2+dm3+dm4+dm5+dm6+dm7+dm8+dm9+dm10+dm11+dm12
    # GDP
    g = self.get_gdp
    gdp = self.USGDP * g[(g.index.month==self.month) & (g.index.year==self.year)]['GDPQXUS'].iloc[-1]
    # Brazil CFR _Fertilizer Weeek
    brazil_cfr_1 = self.BrazilCFR_1 * self.get_BrazilCFR['BrazilCFR'].iloc[-1]
    # Poly. Eq.
    brazil_cfr = fao_2+fao_3+fao_6+eurusd+fertprodquad+g20inflation+monthly_dummy+gdp+brazil_cfr_1+self.const
    print(f"Brazil: ${round(brazil_cfr_1,2)} ->> ${round(brazil_cfr,2)} | {round((brazil_cfr/brazil_cfr_1-1)*100,1)}% {'up' if brazil_cfr >= brazil_cfr_1 else 'down' }")
    return brazil_cfr

In [10]:
fao = self.get_food_price_index
fao['std_8'] = fao['Food Price Index'].rolling(window=8).std()
fao['std_6'] = fao['Food Price Index'].rolling(window=6).std()
fao['std_3'] = fao['Food Price Index'].rolling(window=3).std()

In [13]:
fao

Food Price Index      std_8      std_6      std_3
Date                                                         
1990-01-01         64.145081        NaN        NaN        NaN
1990-02-01         64.453560        NaN        NaN        NaN
1990-03-01         63.787064        NaN        NaN   0.333554
1990-04-01         65.766754        NaN        NaN   1.007296
1990-05-01         64.378874        NaN        NaN   1.016171
...                      ...        ...        ...        ...
2022-03-01        159.713180  10.037283  10.163704  12.644389
2022-04-01        158.435720  11.763876  11.968605  10.414547
2022-05-01        157.885279  12.095372  12.247982   0.937738
2022-06-01        154.255871  11.603364  10.247076   2.271076
2022-07-01        140.935537  10.946680   8.746569   8.924663

[391 rows x 4 columns]